In [48]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import yfinance as yf
import statsmodels.api as sm
import xlwings as xw

In [109]:
TICKERS = ['SPY','AAPL','TSLA']
START_DATE = '2021-01-01'
END_DATE = '2021-11-30'

In [110]:
# get timeseries data from Yahoo Finance

df = yf.download(TICKERS, start=START_DATE, end=END_DATE)

[*********************100%***********************]  3 of 3 completed


In [111]:
# calculate log return

idx = pd.IndexSlice
df_close = df.loc[:,idx['Close']].copy()

df_close_log_return = np.log(df_close) - np.log(df_close.shift(1))
df_close_log_return.dropna(inplace=True)

x = df_close_log_return[['AAPL', 'TSLA']]
y = df_close_log_return[['SPY']]

In [112]:
# run regression model using statmodels

x_con = sm.add_constant(x)
model = sm.OLS(y, x_con).fit()


In [113]:
# print model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    SPY   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     119.0
Date:                Thu, 20 Jan 2022   Prob (F-statistic):           5.46e-36
Time:                        23:34:52   Log-Likelihood:                 864.41
No. Observations:                 228   AIC:                            -1723.
Df Residuals:                     225   BIC:                            -1713.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0006      0.000      1.752      0.0

In [76]:
df_close_pct

,AAPL,SPY,TSLA
Date,,,
2021-01-05,0.012364,0.006887,0.007317
2021-01-06,-0.033662,0.005979,0.028390
2021-01-07,0.034123,0.014857,0.079447
2021-01-08,0.008631,0.005698,0.078403
2021-01-11,-0.023249,-0.006741,-0.078214
...,...,...,...
2021-11-23,0.002422,0.001326,-0.041353
2021-11-24,0.003284,0.002670,0.006285
2021-11-26,-0.031678,-0.022303,-0.030538


In [95]:
df_close_log_return = np.log(df_close) - np.log(df_close.shift(1))

In [98]:
df_close_log_return

,AAPL,SPY,TSLA
Date,,,
2021-01-04,NaN,NaN,NaN
2021-01-05,0.012288,0.006864,0.007291
2021-01-06,-0.034241,0.005961,0.027995
2021-01-07,0.033554,0.014748,0.076448
2021-01-08,0.008594,0.005682,0.075481
...,...,...,...
2021-11-23,0.002419,0.001325,-0.042232
2021-11-24,0.003278,0.002666,0.006265
2021-11-26,-0.032191,-0.022556,-0.031014
